In [1]:
import sys
sys.path.append('../src/')

from docopt import docopt
from Config import Config
from Builder import RepertoireBuilder

doc = """Usage:
    main.py <config_file>

Arguments:
    <config_file>   Path to YAML configuration file
"""


def main():
    config = Config("example.yaml")

    builder = RepertoireBuilder(config)
    return builder.GenerateReportoire()


In [2]:
df = main()


		START


( 0m0s )  -------  1   b2b4   Starting move 
 r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. P . . . . . .
. . . . . . . .
P . P P P P P P
R N B Q K B N R
#CandidateMoves:  7

( 0m2s )  -------  2   e7e5   32% (466256) -- 0.0 -- 49/5/45 
 r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. P . . . . . .
. . . . . . . .
P . P P P P P P
R N B Q K B N R
#CandidateMoves:  0

( 0m5s )  -------  2   d7d5   17% (250025) -- 0.0 -- 48/5/45 
 r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. P . . . . . .
. . . . . . . .
P . P P P P P P
R N B Q K B N R
#CandidateMoves:  0

( 0m9s )  -------  2   g8f6   16% (240359) -- 0.0 -- 46/6/47 
 r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. P . . . . . .
. . . . . . . .
P . P P P P P P
R N B Q K B N R
#CandidateMoves:  0

( 0m13s )  -------  2   e7e6   11% (164234) -- 0.0 -- 48/5/45 
 r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. P . . . . . .
. . . . . . . 

In [3]:
df

,ply,move,IsWhiteToMove,whitePerc,drawPerc,blackPerc,totGames,percGames,engineEval
0,2,e5,True,49,5,45,466256,32,0.00
1,2,d5,True,48,5,45,250025,17,0.00
2,2,Nf6,True,46,6,47,240359,16,0.00
3,2,e6,True,48,5,45,164234,11,0.00
4,2,c6,True,45,6,48,76337,5,0.00
5,3,e4,False,57,5,37,203,0,-0.69
6,4,d5,True,45,5,48,4152,85,-0.20
7,4,e5,True,34,6,59,244,4,-0.52
8,2,a5,True,45,5,49,54691,3,0.12
9,2,g6,True,50,5,43,53436,3,0.16
